# Example 2: Coupled Transmon Qubits

In [1]:
%load_ext autoreload
%autoreload 2
import time

In [2]:
import design as d
import json

with open("design_variables.json") as in_file:
    initial_design_variables = json.load(in_file)


import qdesignoptimizer.utils.names_design_variables as u
from qdesignoptimizer.utils.chip_generation import create_chip_base, ChipType
from qdesignoptimizer.utils.utils import close_ansys
import names as n

## Design assembly

In [4]:
CHIP_NAME = "multi_transmon_chip"
OPEN_GUI = True
chip_type = ChipType(size_x="10mm", size_y="10mm", size_z="-300um")
design, gui = create_chip_base(
    chip_name=CHIP_NAME, chip_type=chip_type, open_gui=OPEN_GUI
)

u.add_design_variables_to_design(design, initial_design_variables)

In [5]:
def render_qiskit_metal_design(design, gui):
    d.add_transmon_plus_resonator(design, nbr=n.NBR_1)
    d.add_transmon_plus_resonator(design, nbr=n.NBR_2)

    d.add_coupler(design)

    d.add_route_interconnects(design)

    d.add_launch_pads(design)

    d.add_chargeline(design, nbr=n.NBR_1)
    d.add_chargeline(design, nbr=n.NBR_2)

    gui.rebuild()
    gui.autoscale()


render_qiskit_metal_design(design, gui)

## MiniStudies

In [7]:
import mini_studies as ms
import optimization_targets as ot
import parameter_targets as pt
import plot_settings as ps

from qdesignoptimizer.design_analysis import DesignAnalysis, DesignAnalysisState

### Single resonator-qubit system
Useful when first tuning up subsystems

In [8]:
MINI_STUDY_NBR = n.NBR_1
MINI_STUDY = ms.get_mini_study_qb_res(nbr=MINI_STUDY_NBR)
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)

opt_targets = ot.get_opt_targets_2qubits_resonator_coupler(
    nbrs=[MINI_STUDY_NBR],
    opt_target_qubit_freq=True,
    opt_target_qubit_anharm=True,
    opt_target_resonator_freq=True,
    opt_target_resonator_kappa=False,
    opt_target_resonator_qubit_chi=True,
)
opt_targets

In [9]:
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)
design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path=CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_parameters=False,
    plot_settings=ps.PLOT_SETTINGS,
    plot_branches_separately=False,
)

nbr_runs = 2  # 10
nbr_passes = 2  # 6
delta_f = 0.001
for i in range(nbr_runs):
    design_analysis.update_nbr_passes(nbr_passes)
    design_analysis.update_delta_f(delta_f)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

[INFO|2025-02-26 11:06:11]: self.eig_solver.sim.setup 
{
    "name": "Resonator_setup",
    "reuse_selected_design": true,
    "reuse_setup": true,
    "min_freq_ghz": 1,
    "n_modes": 1,
    "max_delta_f": 0.5,
    "max_passes": 10,
    "min_passes": 1,
    "min_converged": 1,
    "pct_refinement": 30,
    "basis_order": 1,
    "vars": {
        "Lj": "10 nH",
        "Cj": "0 fF"
    }
}
INFO 11:06AM [connect_project]: Connecting to Ansys Desktop API...
INFO 11:06AM [load_ansys_project]: 	Opened Ansys App
INFO 11:06AM [load_ansys_project]: 	Opened Ansys Desktop v2021.2.0
INFO 11:06AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/lukassp/Documents/Ansoft/
	Project:   Project160
INFO 11:06AM [connect_design]: No active design found (or error getting active design).
INFO 11:06AM [connect]: 	 Connected to project "Project160". No design detected
11:06AM 29s WARNING [activate_ansys_design]: The design_name=get_mini_study_qb_res was not in active project.  Designs in ac

pyaedt INFO: using existing logger.
pyaedt INFO: Launching PyAEDT outside AEDT with CPython and PythonNET.
pyaedt INFO: AEDT installation Path C:\Program Files\AnsysEM\AnsysEM21.2\Win64.
pyaedt INFO: Launching AEDT with module PythonNET.
pyaedt INFO: Ansoft.ElectronicsDesktop.2021.2 Started with process ID 26464.
pyaedt INFO: pyaedt v0.6.46
pyaedt INFO: Python version 3.10.15 | packaged by Anaconda, Inc. | (main, Oct  3 2024, 07:22:19) [MSC v.1929 64 bit (AMD64)]
pyaedt INFO: No project is defined. Project Project160 exists and has been read.
pyaedt INFO: Active Design set to get_mini_study_qb_res
pyaedt INFO: Aedt Objects initialized
pyaedt INFO: Variable Manager initialized
pyaedt INFO: Design Loaded
pyaedt INFO: Successfully loaded project materials !
pyaedt INFO: Materials Loaded


[INFO|2025-02-26 11:06:42]: No fine mesh map was found for name_qubit_1
[INFO|2025-02-26 11:06:42]: No fine mesh map was found for name_resonator_1
[INFO|2025-02-26 11:06:42]: No fine mesh map was found for name_tee1
INFO 11:06AM [analyze]: Analyzing setup Setup


Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1
Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1
Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1
Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1
Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1

        energy_elec_all       = 8.42027652519007e-25
        energy_elec_substrate = 7.37924146200782e-25
        EPR of substrate = 87.6%

        energy_mag    = 1.14874006852569e-26
        energy_mag % of energy_elec_all  = 1.4%
        

Variation 0  [1/1]

  Mode 0 at 4.90 GHz   [1/2]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
               98.6%   4.21e-25 5.744e-27

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_0j   sign s_0j    (p_capacitive)
		Energy fraction (Lj over Lj&Cj)= 98.12%
	jj_name_qub

WARNING 11:07AM [__init__]: <p>Error: <class 'IndexError'></p>


...

Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1
Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1
Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1
Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1
Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1

        energy_elec_all       = 1.51528945610287e-23
        energy_elec_substrate = 1.40300515614229e-23
        EPR of substrate = 92.6%

        energy_mag    = 1.51259833557992e-23
        energy_mag % of energy_elec_all  = 99.8%
        

Variation 0  [1/1]

  Mode 0 at 4.09 GHz   [1/2]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
               98.9%  5.855e-23 6.383e-25

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_0j   sign s_0j    (p_capacitive)
		Energy fraction (Lj over Lj&Cj)= 98.15%
	jj_name_qu

WARNING 11:07AM [__init__]: <p>Error: <class 'IndexError'></p>



ANALYSIS DONE. Data saved to:

C:\data-pyEPR\Project160\get_mini_study_qb_res\2025-02-26 11-07-31.npz


	 Differences in variations:



 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Variation 0

Starting the diagonalization
Finished the diagonalization
Pm_norm=
modes
0    1.020425
1    1.022991
dtype: float64

Pm_norm idx =
   jj_name_qubit_1
0             True
1            False
*** P (participation matrix, not normlz.)
   jj_name_qubit_1
0         0.969389
1         0.001772

*** S (sign-bit matrix)
   s_jj_name_qubit_1
0                 -1
1                 -1
*** P (participation matrix, normalized.)
      0.99
    0.0018

*** Chi matrix O1 PT (MHz)
    Diag is anharmonicity, off diag is full cross-Kerr.
       179    0.973
     0.973  0.00132

*** Chi matrix ND (MHz) 
       197    0.844
     0.844 0.000999

*** Frequencies O1 PT (MHz)
0    3909.891734
1    6211.837098
dtype: float64

*** Frequencies ND (MHz)
0    3901.158854
1    6211.854086
d

[INFO|2025-02-26 11:07:35]: freq_ND_results
{
    "0": {
        "0": 3901.1588535862916,
        "1": 6211.854086107633
    }
}


0 <qdesignoptimizer.sim_plot_progress.OptPltSet object at 0x00000273018A4070> [1, 2] [6664527609.47263, 6211854086.107633]
1 <qdesignoptimizer.sim_plot_progress.OptPltSet object at 0x00000273018A7EE0> [1, 2] [53057.339439, 109053.1355476]
2 <qdesignoptimizer.sim_plot_progress.OptPltSet object at 0x00000273018A7E80> [1, 2] [None, None]
0 <qdesignoptimizer.sim_plot_progress.OptPltSet object at 0x00000273018A7E20> [1, 2] [4710906199.265343, 3901158853.586292]
1 <qdesignoptimizer.sim_plot_progress.OptPltSet object at 0x00000273018A7DC0> [1, 2] [196077045.3236351, 197389371.22040272]
0 <qdesignoptimizer.sim_plot_progress.OptPltSet object at 0x00000273018A7D60> [1, 2] [597453.020784378, 843659.9167690276]


In [ ]:
design_analysis.overwrite_parameters()

### Two qubit-resonator system

In [ ]:
MINI_STUDY = ms.get_mini_study_2qb_resonator_coupler()
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)

opt_targets = ot.get_opt_targets_2qubits_resonator_coupler(
    nbrs=[n.NBR_1, n.NBR_2],
    opt_target_qubit_freq=True,
    opt_target_qubit_anharm=True,
    opt_target_resonator_freq=True,
    opt_target_resonator_kappa=False,
    opt_target_resonator_qubit_chi=True,
    opt_target_coupler_freq=True,
)

In [ ]:
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)
design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path=CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_parameters=False,
    plot_settings=ps.PLOT_SETTINGS,
    plot_branches_separately=False,
)

nbr_runs = 2
nbr_passes = 3
delta_f = 0.001
for i in range(nbr_runs):
    design_analysis.update_nbr_passes(nbr_passes)
    design_analysis.update_delta_f(delta_f)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

In [ ]:
design_analysis.overwrite_parameters()

### Purcell decay into charge line

The T1 of the qubit is limited by its decay into the charge line. This ministudy will change the distance of the charge line from the transmon pocket to optimize a T1 limit of 1 ms. This value is computed with simple capacitance matrix simulations and a classical model, assuming that the qubit frequency has already been optimized.

In [ ]:
MINI_STUDY_BRANCH = 0
MINI_STUDY = ms.get_mini_study_qb_charge_line(branch=MINI_STUDY_BRANCH)
opt_targets = ot.get_opt_targets_qb_charge_line(branch=MINI_STUDY_BRANCH)
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)

In [ ]:
# %matplotlib inline
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.TARGET_PARAMS_CHARGE_LINE_DECAY
)
design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path=CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_parameters=False,
    plot_settings=ps.PLOT_SETTINGS_CHARGE_LINE_DECAY,
    plot_branches_separately=False,
)

In [ ]:
nbr_runs = 3
nbr_passes_cap = 2

for i in range(nbr_runs):
    design_analysis.update_nbr_passes_capacitance_ministudies(nbr_passes_cap)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

## View Optimization results

In [ ]:
design_analysis.get_cross_kerr_matrix(iteration=-1)

In [ ]:
close_ansys()